# CLI

> Create and load pre-built aimagic notebooks for contexts

In [ ]:
#| default_exp cli

In [ ]:
#|export
import webbrowser
from urllib.parse import quote
from pathlib import Path
import ContextKit.contexts as ctx
import argparse
from fastcore.all import bind
from ContextKit.projects import *

In [ ]:
#| export
def open_nb(path=None, host='localhost', port=8888, base_dir='~/github'):
    base_url = f"http://{host}:{port}/notebooks"
    base_dir = Path(base_dir).expanduser()
    current_dir = str(Path.cwd().relative_to(base_dir))
    url = f"{base_url}/{quote(current_dir)}"
    if path:
        url += f"/{quote(str(path))}"
    webbrowser.open(url)
    

In [ ]:
#| export
def create_nb_cmd(path, ctx_name, force, templates):
    if path.is_dir():
        print(f"Error: {path} is a directory. Please provide a file path.")
        return
    if ctx_name not in templates:
        print(f"Error: {ctx_name} is not a valid context name.")
        print(f"Valid options are: {', '.join(templates.keys())}")
        return
    if path.exists() and not force:
        overwrite = input(f"{path} already exists. Overwrite? (y/N): ").lower() == 'y'
        if not overwrite:
            print("Create NB cancelled.  If if using `co`, will open existing nb.")
            return
    create_magic_template_nb(ctx_name, str(path))
    print(f"Notebook created at {path}")

In [ ]:
#| export
def create_and_open_nb(path, ctx_name, force, templates,
                       host='localhost', port=8888, base_dir=None):
    create_nb_cmd(path, ctx_name, force, templates)
    open_nb(path, host, port, base_dir)

In [ ]:
#| export    
def ck():
    templates = {name[4:]: name for name in dir(ctx) if name.startswith('ctx_')}
    
    parser = argparse.ArgumentParser(description="Create a magic template notebook or open Jupyter notebook.")
    subparsers = parser.add_subparsers(dest='command')

    create_parser = subparsers.add_parser('c', help='Create a magic template notebook')
    create_parser.add_argument('ctx_name', help=f"Context name. One of: {', '.join(templates.keys())}")
    create_parser.add_argument('path', help="Output path for the notebook")
    create_parser.add_argument('--force', default=False, action='store_true', help="Force overwrite without prompting")
    
    jnb_parser = subparsers.add_parser('o', help='Open Jupyter notebook')
    jnb_parser.add_argument('path', help="Path to the notebook")
    jnb_parser.add_argument('--host', default='localhost', help="Jupyter server host")
    jnb_parser.add_argument('--port', type=int, default=8888, help="Jupyter server port")
    jnb_parser.add_argument('--jupy-base-dir', type=Path, default='~/github', help="Base directory for notebook paths")

    create_and_open_parser = subparsers.add_parser('co', help='Create a magic template notebook and open it')
    create_and_open_parser.add_argument('ctx_name', help=f"Context name. One of: {', '.join(templates.keys())}")
    create_and_open_parser.add_argument('path', help="Output path for the notebook")
    create_and_open_parser.add_argument('--force', default=False, action='store_true', help="Force overwrite without prompting")
    create_and_open_parser.add_argument('--host', default='localhost', help="Jupyter server host")
    create_and_open_parser.add_argument('--port', type=int, default=8888, help="Jupyter server port")
    create_and_open_parser.add_argument('--jupy_base-dir', type=Path, default='~/github', help="Base directory for notebook paths")

    args = parser.parse_args()
    
    args.path = Path(args.path).with_suffix('.ipynb')

    if args.command == 'c': create_nb_cmd(args.path, args.ctx_name, args.force, templates)
    elif args.command == 'o': open_nb(args.path, args.host, args.port, args.jupy_base_dir)
    elif args.command == 'co': create_and_open_nb(args.path, args.ctx_name, args.force, templates,
                                                  args.host, args.port, args.jupy_base_dir)
    else: parser.print_help()